# Task Description

Задача: генерация овета по вопросу
<br>
Пример:
<br>
Input: что почитать? мне 10.
<br>
Ответ модели: почитай стивена кинга

# Required Imports

In [ ]:
# !pip install youtokentome

In [4]:
import math
import re
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from tqdm.auto import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

# Data

In [5]:
data = open('train.txt').readlines()

In [6]:
questions = [row.replace('\n', '').split('\t')[0] for row in data]

In [7]:
answers = [row.replace('\n', '').split('\t')[1] for row in data]

Берем 1 млн. пар вопрос-ответ для обучения

In [8]:
questions = questions[0:1_000_000]
answers = answers[0:1_000_000]

Пример данных:

In [9]:
for q,a in zip(questions[46:50], answers[46:50]):
  print(f'q: {q}\na: {a}')

q: а вы часто едете по встречной полосе.или чаще как все?)
 a: на дорогах я соблюдаю правила; но не в жизни:)
q: что можно маме подарить
 a: оренбургский пуховый платок.
q: а тишина может быть яркой ?
 a: может быть полной
q: от вашего имени. какие можно соорудить производные)?
 a: дианка-диванка, дианчик, ди


# BPE tokenization

In [10]:
pad_index = 0 # pad-индекс
bos_index = 2 # индекс начала предложения
eos_index = 3 # индекс конца предложения

vocab_size = 30_000 # размер словаря

"Обучаем" токенизатор на вопросах и ответах из датасета:


In [11]:
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
  for q,a in zip(questions, answers):
      f.write(q + '\n')
      f.write(a + '\n')

model_path = 'pretrained_bpe_lm.model'

yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)


In [12]:
def bpe_tokenization(texts):
  """BPE tokenizer"""
  
  tokenizer = yttm.BPE(model=model_path)      

  tokenized = []

  batch_size = 256

  for i_batch in tqdm(range(math.ceil(len(texts) / batch_size))):
      
      tokenized.extend(tokenizer.encode(
          list(texts[i_batch*batch_size:(i_batch+1)*batch_size]), bos=False, eos=False))
      
  return tokenized      

In [13]:
tokenized_questions = bpe_tokenization(questions)

  0%|          | 0/3907 [00:00<?, ?it/s]

In [14]:
tokenized_responses = bpe_tokenization(answers)

  0%|          | 0/3907 [00:00<?, ?it/s]

## Dataset

In [15]:
class SeqToSeqDataset(torch.utils.data.Dataset):

    def __init__(self, tokenized_questions, tokenized_responses):
        
        self.questions = tokenized_questions
        self.responses = tokenized_responses

    def __len__(self):
        return len(self.questions)

    def padding(self, tokenized):
      max_batch_length = 0 # максимальная длина последовательности в батче

      for sequence in tokenized:
        max_batch_length = max(max_batch_length, len(sequence))
      
      for n_sequence in range(len(tokenized)):
        pads = [pad_index] * (max_batch_length - len(tokenized[n_sequence]))
        tokenized[n_sequence] = [bos_index] + tokenized[n_sequence] + [eos_index] +  pads

      return tokenized

    def __getitem__(self, index):
        batch_qestions = torch.tensor(self.padding(self.questions[index])).long() # вопросы
        responses = torch.tensor(self.padding(self.responses[index])).long() 
        batch_responses = responses[:, :-1]
        batch_true = responses[:, 1:]
        
        return batch_qestions, batch_responses, batch_true

In [16]:
batch_size = 256

batches_questions = [] # батчи с вопросами

for i_batch in range(math.ceil(len(tokenized_questions) / batch_size)):
    
    batches_questions.append(tokenized_questions[i_batch*batch_size:(i_batch+1)*batch_size])

In [17]:
batch_size = 256

batches_responses = [] # батчи с ответами

for i_batch in range(math.ceil(len(tokenized_responses) / batch_size)):
    
    batches_responses.append(tokenized_responses[i_batch*batch_size:(i_batch+1)*batch_size])

In [ ]:
len(batches_questions) # число батчей

In [18]:
assert len(batches_questions) == len(batches_responses)

In [19]:
validation_start_index = int(len(batches_questions) * 0.05) # индекс начала валидационной выборки (95/5)

In [20]:
train_dataset = SeqToSeqDataset(batches_questions[:-validation_start_index], batches_responses[:-validation_start_index])
validation_dataset = SeqToSeqDataset(batches_questions[-validation_start_index:], batches_responses[-validation_start_index:])

In [21]:
def collate_fn(x):
    """remove extra-dimention"""
    
    x, y, z = x[0]

    return x, y, z

Оборачиваем данные в DataLoader:

In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=1,
                                           collate_fn=collate_fn,
                                           shuffle=False)

validation_loader = torch.utils.data.DataLoader(validation_dataset,
                                                batch_size=1,
                                                collate_fn=collate_fn,
                                                shuffle=False)

Проверка итератора

In [23]:
progress_bar = tqdm(total=len(validation_loader.dataset), desc='Testing')

for x, y, z in validation_loader:
    progress_bar.update()
    
progress_bar.close()

Testing:   0%|          | 0/195 [00:00<?, ?it/s]

In [ ]:
x

In [ ]:
y

In [ ]:
z

## Encoder-Decoder Blocks

In [24]:
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=512) # один слой эмбеддингов для энкодера и декодера

In [25]:
class Encoder(nn.Module):
  """Encoder block"""
  
  def __init__(self, embedding_layer):
    super().__init__()

    self.embedding_layer = embedding_layer # слой эмбеддингов

    self.backbone = nn.LSTM(input_size=512, hidden_size=512, num_layers=3, batch_first=True, dropout=0.25) # 3-х слойная LSTM

  def forward(self, x):

    x_emb = self.embedding_layer(x)

    x_backbone, memory = self.backbone(x_emb)

    return x_backbone, memory

In [26]:
class Decoder(nn.Module):
  """Decoder block"""

  def __init__(self, embedding_layer):
    super().__init__()

    self.embedding_layer = embedding_layer

    self.backbone = nn.LSTM(input_size=512, hidden_size=512, num_layers=3, batch_first=True, dropout=0.25)

  def forward(self, x, encoder_memory):

    x_emb = self.embedding_layer(x)

    x_backbone, _ = self.backbone(x_emb, encoder_memory)

    return x_backbone

In [27]:
class EncoderDecoder(nn.Module):
  """Encoder-Decoder Model"""

  def __init__(self, encoder, decoder):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder

    self.lm_head = nn.Linear(512, vocab_size) # вероятность следующего токена, выбираем из 55К

  def forward(self, encoder_sequence, decoder_sequence):
    _, memory = self.encoder(encoder_sequence)
    decoder_hidden = self.decoder(decoder_sequence, memory)

    logits = self.lm_head(decoder_hidden) 

    return logits

# Train & Evaluate logic

In [28]:
def train(model, loader, criterion, optimizer, clip=3., last_n_losses=500, verbose=True):
    
    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')

    model.train()

    for q, x, y in loader:

        q = q.to(device)
        x = x.to(device)
        y = y.to(device)

        logits = model(q,x) 

        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
        
        optimizer.zero_grad()
        loss.backward()
        # рассчитали градиенты и клипаем их
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))

        progress_bar.update()

    progress_bar.close()
    
    return losses

In [29]:
# фукнция для расчета метрик на тесте

def evaluate(model, loader, criterion, last_n_losses=500, verbose=True):
    
    losses = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Evaluate')

    model.eval()

    for q, x, y in loader:

        q = q.to(device)
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
            logits = model(q, x)

        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))

        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))

        progress_bar.update()

    progress_bar.close()
    
    return losses

# Model Initialization

In [30]:
encoder = Encoder(embedding_layer)

In [31]:
decoder = Decoder(embedding_layer)

In [32]:
model = EncoderDecoder(encoder, decoder)

In [33]:
device = torch.device('cuda:0')

In [34]:
model.to(device)

EncoderDecoder(
  (encoder): Encoder(
    (embedding_layer): Embedding(30000, 512)
    (backbone): LSTM(512, 512, num_layers=3, batch_first=True, dropout=0.25)
  )
  (decoder): Decoder(
    (embedding_layer): Embedding(30000, 512)
    (backbone): LSTM(512, 512, num_layers=3, batch_first=True, dropout=0.25)
  )
  (lm_head): Linear(in_features=512, out_features=30000, bias=True)
)

In [35]:
optimizer = torch.optim.Adam(params=model.parameters())

In [36]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
print(f'Количество обучаемых параметров в сети: {count_parameters(model):,}')

Количество обучаемых параметров в сети: 43,357,488


# Model Training

В данной версии ноутбука модель обучалась на GPU 2 эпохи в течение 40 минут. У меня был чуть более качественный вариант модели (где я брала побольше данных и обучала дольше), но пришлось перезапустить обучение. Поэтому для сокращения времени взяла поменьше данных и только две эпохи

In [40]:
# задайте сколько вам комфортно обучать модель по времени
# в идеале пару часов

epochs = 2 

train_losses = []
validation_losses = []

train_perplexities = []
validation_perplexities = []

best_validation_loss = 1e+6

for n_epoch in range(1, epochs + 1):
    
    epoch_train_losses = train(model, train_loader, criterion, optimizer)
    epoch_validation_losses = evaluate(model, validation_loader, criterion)
    
    mean_train_loss = np.mean(epoch_train_losses)
    mean_validation_loss = np.mean(epoch_validation_losses)
    
    train_losses.append(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    validation_losses.append(epoch_validation_losses)
    validation_perplexities.append(np.exp(mean_validation_loss))
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'
    message += f'Validation: loss - {mean_validation_loss:.4f} | perplexity - {validation_perplexities[-1]:.3f}'
    
    print(message)
    
    if mean_validation_loss < best_validation_loss:
        
        best_validation_loss = mean_validation_loss
        
        torch.save(model.state_dict(), f'best_language_model_state_dict.pth')
        torch.save(optimizer.state_dict(), 'best_optimizer_state_dict.pth')
        
    else:
        break
        
    torch.save(model.state_dict(), f'last_language_model_state_dict.pth')    
    torch.save(optimizer.state_dict(), 'last_optimizer_state_dict.pth')

    with open(f'info_{n_epoch}.json', 'w') as file_object:

        info = {
            'message': message,
            'train_losses': train_losses,
            'validation_losses': validation_losses,
            'train_perplexities': train_perplexities,
            'validation_perplexities': validation_perplexities
        }

        file_object.write(json.dumps(info, indent=2))

Train:   0%|          | 0/3712 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/195 [00:00<?, ?it/s]

Epoch: 1
Train: loss - 6.6433 | perplexity - 767.638
Validation: loss - 6.0588 | perplexity - 427.859


Train:   0%|          | 0/3712 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/195 [00:00<?, ?it/s]

Epoch: 2
Train: loss - 4.8769 | perplexity - 131.227
Validation: loss - 4.7059 | perplexity - 110.594


In [41]:
model.load_state_dict(torch.load('best_language_model_state_dict.pth'))

<All keys matched successfully>

In [42]:
tokenizer = yttm.BPE(model=model_path) 

# Generate Answers

Генерация ответа методом greedy search:

In [43]:
def generate(model, question, bos_index=2, eos_index=3, max_sequence=25):
    """Greedy Search sampling"""
    
    # токенизируем вопрос
    question_tokenized = tokenizer.encode(question, bos=False, eos=False)    
    question_tokenized = [bos_index] + question_tokenized + [eos_index]
    question_tokenized = torch.tensor(question_tokenized).long().to(device)

    # в качестве начальной последовательности для декодера подаем токен BOS
    decoder_sequence =  torch.tensor([bos_index]).long().to(device)
    
    model.eval()

    with torch.no_grad():
      
      # получаем последнее скрытое состояние из энкодера
      _, memory = model.encoder(question_tokenized)

      pred = []

      for timestamp in range(max_sequence):
        decoder_hidden = model.decoder(decoder_sequence, memory)

        # получаем предсказания для следующего токена
        next_token_prediction = model.lm_head(decoder_hidden) 
        pred.append(next_token_prediction)

        next_token_prediction = next_token_prediction[-1]
        current_token = next_token_prediction.argmax(dim=0).item()

        curr_tensor = torch.tensor([current_token]).long().to(device)
        decoder_sequence = torch.cat((decoder_sequence, curr_tensor),dim=-1)

        # останавливаем генерировать текст, когда встретили токен конца предложения
        if current_token == eos_index:
            break

    answer = tokenizer.decode(decoder_sequence.tolist())
    answer = [re.sub('<BOS>', '', _) for _ in answer][0].replace('<EOS>', '')

    return(answer)

In [74]:
questions = [
    'сколько стоит книга?',
    'какое самое вкусное блюдо в мире?',
    'какой самый лучший город для комфортной жизни?',
    'почему нет снега?',
    'как научиться играть на гитаре?',
    'за сколько дней можно написать книгу с нуля?',
    'как дела?'
]

При обучении модели на большем объеме данных (2 млн. пар), ответы получались более качественными и разнообразными. Примеры ответов для модели, обученной на ~1 млн. примеров:

In [75]:
for q in questions:
  print(f'— {q}\n— {generate(model, q)}\n')

— сколько стоит книга?
—  от 2000 руб.

— какое самое вкусное блюдо в мире?
—  с мясом

— какой самый лучший город для комфортной жизни?
—  в москве - в москве

— почему нет снега?
—  потому что это не лечится.

— как научиться играть на гитаре?
—  в аптеке

— за сколько дней можно написать книгу с нуля?
—  за 10 лет

— как дела?
—  не знаю

